## Dependencies

In [1]:
import prettytable as pt
import json
import pandas as pd
from tqdm import tqdm
import csv
import os
import random
import re

## Global Variables

### Data Directory

In [22]:
dataDir = "../Data"

### Some data about two large datasets

In [23]:
totalReviews = 157260921
totalReviewPath = os.path.join(dataDir, "All_Amazon_Review_5.json")

totalMeta = 15023060
totalMetaPath = os.path.join(dataDir, "All_Amazon_Meta.json")

## Simple Analysis

### Review dataset

In [108]:
# simple analysis on review dataset
totalReviews = 157260921
columns = ['verified', 'image', 'style', 'asin', 'reviewerID', 'overall', 'reviewText', 'reviewTime', 'unixReviewTime', 'summary', 'reviewerName', 'vote']
column_dict = dict()
style = set()
dataPath = totalReviewPath
with open(dataPath, 'r', encoding="utf-8") as f:
    with tqdm(total=totalReviews, desc="Processing", leave=True, unit_scale=True) as pbar:
        index = 0
        line = f.readline()
        while line:
            pbar.update(1)
            row = json.loads(line)
            for key in row.keys():
                valueType = type(row[key])
                if key not in column_dict:
                    if valueType in [list, dict, str]:
                        column_dict[key] = (valueType.__name__, index, len(row[key]))
                    else:
                        column_dict[key] = (valueType.__name__)
                else:
                    if valueType in [list, dict, str] and len(row[key]) > column_dict[key][2]:
                        column_dict[key] = (valueType.__name__, index, len(row[key]))
            line = f.readline()
            index += 1
print(column_dict)

Processing: 100%|█████████▉| 157M/157M [30:48<00:00, 85.1kit/s]   

{'overall': 'float', 'verified': 'bool', 'reviewTime': ('str', 1, 11), 'reviewerID': ('str', 11586, 20), 'asin': ('str', 0, 10), 'reviewerName': ('str', 28782337, 1725), 'reviewText': ('str', 140467823, 35094), 'summary': ('str', 151619415, 1730), 'unixReviewTime': 'int', 'vote': ('str', 257028, 6), 'image': ('list', 136488683, 508), 'style': ('dict', 2377009, 7)}


### Metadata

In [25]:
# simple analysis on review dataset
totalMeta = 15023060
column_dict = dict()
style = set()
dataPath = totalMetaPath
with open(dataPath, 'r', encoding="utf-8") as f:
    with tqdm(total=totalMeta, desc="Processing", leave=True, unit_scale=True) as pbar:
        for index, line in enumerate(f):
            row = json.loads(line)
            for key in row.keys():
                valueType = type(row[key])
                if key not in column_dict:
                    if valueType in [list, dict, str]:
                        column_dict[key] = (valueType.__name__, index, len(row[key]))
                    else:
                        column_dict[key] = (valueType.__name__)
                else:
                    if valueType in [list, dict, str] and len(row[key]) > column_dict[key][2]:
                        column_dict[key] = (valueType.__name__, index, len(row[key]))
            pbar.update(1)
print(column_dict)

Processing: 100%|█████████▉| 15.0M/15.0M [15:49<00:00, 15.8kit/s]

{'category': ('list', 14000052, 171), 'tech1': ('str', 11042410, 25479), 'description': ('list', 5503049, 2253), 'fit': ('str', 14567591, 3701), 'title': ('str', 144162, 6234), 'also_buy': ('list', 1501, 100), 'image': ('list', 14728, 50), 'tech2': ('str', 7681322, 10348), 'brand': ('str', 7874918, 2000), 'feature': ('list', 14000052, 165), 'rank': ('str', 12046222, 1756), 'also_view': ('list', 58, 60), 'details': ('dict', 6534924, 12), 'main_cat': ('str', 1196634, 190), 'similar_item': ('str', 2289714, 84480), 'date': ('str', 10848403, 1264), 'price': ('str', 147, 4648), 'asin': ('str', 0, 10)}


## Split Review Columns

### Single Process

In [40]:
def splitColumns(selectedKeys:list, dataPath:str, totalRowNum:int, outputPath:str):
    """function used to split dataset

    Args:
        selectedKeys (list): list contains column names
        dataPath (str): input file path
        totalRowNum (int): number of rows
        outputPath (str): output file path
    """
    with open(outputPath, "w", newline="") as outputFile:
        writer = csv.DictWriter(outputFile, selectedKeys)
        writer.writeheader()
        with open(dataPath, encoding="utf-8") as file:
            with tqdm(total=totalRowNum, desc="Processing", leave=True, unit_scale=True) as pbar:
                for line in file:
                    row = json.loads(line)
                    writer.writerow(dict([(key, row.get(key)) for key in selectedKeys]))
                    pbar.update()

totalReviews = 157260921 # total number of reviews
dataPath = "../Data/All_Amazon_Review_5.json" # input dataset path
outputPath = "../Data/All_Amazon_Review_User_Item_Rating.csv" # output datset path
selectedKeys = ['reviewerID', 'asin', 'overall'] # choose columns to split from dataset
splitColumns(selectedKeys, dataPath, totalReviews, outputPath)

Processing: 100%|█████████▉| 157M/157M [23:15<00:00, 113kit/s]  


### Multi-process

In [8]:
import multiprocess_methods
from multiprocessing import cpu_count
if __name__ == '__main__':
    dataPath = os.path.join(dataDir, "All_Amazon_Review_5.json")
    totalSize = 157260921
    selectedKeys = ['reviewerID', 'asin', 'overall']
    outputPath = os.path.join(dataDir, "All_Amazon_Review_User_Item_Rating.csv")
    processNum = cpu_count() # number of processes (customize this variable based on different CPUs)
    chunkSize = 1024 # the size of each chunk splitted from the iterable
    printParameters = True
    multiprocess_methods.splitDataset(dataPath, totalSize, selectedKeys, outputPath, processNum, chunkSize, printParameters)

Dataset File: ../Data\All_Amazon_Review_5.json
Total Size: 157260921
Selected Keys: ['reviewerID', 'asin', 'overall']
Output File: ../Data\All_Amazon_Review_User_Item_Rating.csv
Number of processes: 12
Size of each chunk: 1024


Processing: 100%|█████████▉| 157M/157M [19:50<00:00, 132kit/s]    


## Split Metadata

### Analysis on category

In [43]:
category_list = []

dataPath = "../Data/All_Amazon_Meta.json"
totalRowNum = 15023060

with tqdm(total=totalRowNum, desc="Processing", leave=True, unit_scale=True) as pbar:
    with open(dataPath, 'r', encoding="utf-8") as file:
            for index, line in enumerate(file):
                row = json.loads(line)
                if row['main_cat'].lower() not in category_list:
                    alt = re.search("alt\s*=\s*\"(.*)\"", row['main_cat'].lower())
                    if alt and alt.group(1) not in category_list:
                        category_list.append(alt.group(1))
                        # print(alt.group(1))
                    elif not alt:
                        category_list.append(row['main_cat'].lower())
                        # print(row['main_cat'].lower())                             
                pbar.update()
print(category_list)

Processing: 100%|█████████▉| 15.0M/15.0M [10:53<00:00, 23.0kit/s]

['movies & tv', 'amazon fashion', 'sports & outdoors', 'amazon home', 'health & personal care', 'books', 'toys & games', 'baby', 'office products', 'all beauty', 'arts, crafts & sewing', 'digital music', 'video games', 'home audio & theater', 'tools & home improvement', 'all electronics', 'cell phones & accessories', 'camera & photo', 'industrial & scientific', 'musical instruments', 'automotive', 'computers', 'grocery', 'portable audio & accessories', 'software', 'car electronics', 'pet supplies', 'appliances', 'collectible coins', '', 'entertainment', 'luxury beauty', 'gps & navigation', 'amazon devices', 'buy a kindle', 'amazon launchpad', 'handmade', 'gift cards', 'sports collectibles', 'fine art', 'memberships & subscriptions', 'health &amp; personal care', 'arts, crafts &amp; sewing', 'toys &amp; games', 'industrial &amp; scientific', 'sports &amp; outdoors', 'tools &amp; home improvement', 'cell phones &amp; accessories', 'audible audiobooks', 'magazine subscriptions', 'camera &

### Single process

In [41]:
def splitMetaColumns(category_dict:dict, selectedKeys:list, dataPath:str, totalRowNum:int, outputPath:str, printParameters:bool = False):
    """function used to split dataset

    Args:
        selectedKeys (list): list contains column names
        dataPath (str): input file path
        totalRowNum (int): number of rows
        outputPath (str): output file path
    """
    if printParameters:
        # print(f"Category List; {category_dict}")
        print(f"Dataset File: {dataPath}")
        print(f"Total Size: {totalRowNum}")
        print(f"Selected Keys: {selectedKeys}")
        print(f"Output Directory: {outputPath}")

    if not os.path.exists(outputPath):
        os.mkdir(outputPath)

    for category in category_dict.keys():
        fileName = category + ".csv"
        filePath = os.path.join(outputPath, fileName)
        with open(filePath, "w", newline="") as outputFile:
            writer = csv.DictWriter(outputFile, selectedKeys)
            writer.writeheader()
    
    with open(dataPath, encoding="utf-8") as file:
        with tqdm(total=totalRowNum, desc="Processing", leave=True, unit_scale=True) as pbar:
            for line in file:
                row = json.loads(line)
                rowDict = dict([(key, row.get(key)) for key in selectedKeys])
                alt = re.search("alt\s*=\s*\"(.*)\"", row['main_cat'].lower())
                if alt:
                    rowDict['main_cat'] = alt.group(1)
                else:
                    rowDict['main_cat'] = rowDict['main_cat'].lower()
                
                category = ""
                for key, values in category_dict.items():
                    if rowDict['main_cat'] in values:
                        category = key
                        break
                if category == "":
                    print(row['main_cat'])
                    print(row['title'])
                    break
                with open(os.path.join(outputPath, category + ".csv"), "a", newline="") as outputFile:
                    writer = csv.DictWriter(outputFile, selectedKeys)
                    writer.writerow(rowDict)
                pbar.update()

category_dict = {
    "empty": [
        ""
    ],
    "amazon product": [
        "alexa skills",
        "amazon devices",
        "amazon fashion",
        "amazon fire tv",
        "amazon home",
        "amazon launchpad",
        "fire phone",
        "handmade",
        "memberships & subscriptions"
    ],
    "all beauty": [
        "all beauty"
    ],
    "appliances": [
        "appliances"
    ],
    "arts crafts and sewing": [
        "arts, crafts & sewing",
        "arts, crafts &amp; sewing"
    ],
    "automotive": [
        "automotive",
        "vehicles"
    ],
    "books": [
        "audible audiobooks",
        "books"
    ],
    "cell phones and accessories": [
        "cell phones & accessories",
        "cell phones &amp; accessories"
    ],
    "clothing shoes and jewelry": [
        "shorts"
    ],
    "collectibles and fine art": [
        "collectible coins",
        "collectibles & fine art",
        "fine art",
        "sports collectibles"
    ],
    "digital music": [
        "digital music"
    ],
    "electronics": [
        "all electronics",
        "apple products",
        "beats by dr. dre",
        "camera & photo",
        "camera &amp; photo",
        "car electronics",
        "computers",
        "gps & navigation",
        "gps &amp; navigation",
        "portable audio & accessories",
        "portable audio &amp; accessories"
    ],
    "gift cards": [
        "gift cards"
    ],
    "grocery and gourmet food": [
        "grocery"
    ],
    "home and kitchen": [
        "baby",
        "health & personal care",
        "health &amp; personal care",
        "home & business services",
        "home audio & theater",
        "home audio &amp; theater"
    ],
    "industrial and scientific": [
        "3d printing",
        "industrial & scientific",
        "industrial &amp; scientific"
    ],
    "kindle store": [
        "buy a kindle"
    ],
    "luxury beauty": [
        "luxury beauty"
    ],
    "magazine subscriptions": [
        "magazine subscriptions"
    ],
    "movies and tv": [
        "movies & tv",
        "movies &amp; tv"
    ],
    "musical instruments": [
        "musical instruments"
    ],
    "office products": [
        "office products"
    ],
    # "patio lawn and garden": [
        
    # ],
    "pet supplies": [
        "pet supplies"
    ],
    "prime pantry": [
        "prime pantry"
    ],
    "software": [
        "software"
    ],
    "sports and outdoors": [
        "sports & outdoors",
        "sports &amp; outdoors"
    ],
    "tools and home improvement": [
        "tools & home improvement",
        "tools &amp; home improvement"
    ],
    "toys and games": [
        "toys & games",
        "toys &amp; games"
    ],
    "video games": [
        "video games"
    ],
    "entertainment":[
        "entertainment"
    ]
}

totalMeta = 15023060
dataPath = "../Data/All_Amazon_Meta.json" # input dataset path
outputPath = "../Data/Metadata/" # output datset path
selectedKeys = ['main_cat', 'asin', 'title'] # choose columns to split from dataset
splitMetaColumns(category_dict, selectedKeys, dataPath, totalMeta, outputPath, True)

Dataset File: ../Data/All_Amazon_Meta.json
Total Size: 15023060
Selected Keys: ['main_cat', 'asin', 'title']
Output Directory: ../Data/Metadata/


Processing: 100%|█████████▉| 15.0M/15.0M [43:18<00:00, 5.78kit/s]  


In [2]:
train_df = pd.read_csv("../Data/train_100k.csv")
train_df

,verified,asin,reviewerID,overall,reviewText,unixReviewTime,summary,vote
0,True,B00UO1PFQO,A0038036HJMMW88H5SCN,1.0,"It will not attach as said in instructions, wi...",1494633600,"It will not attach as said in instructions, wi...",4
1,True,B00WA4AM7U,A0038036HJMMW88H5SCN,5.0,"Great fit, like very much!",1494633600,Five Stars,0
2,True,B00WA4AM7U,A0038036HJMMW88H5SCN,5.0,"Great fit, like very much!",1494633600,Five Stars,0
3,True,B00WA4AM7U,A0038036HJMMW88H5SCN,5.0,"Great fit, like very much!",1494633600,Five Stars,0
4,True,B00WA4AM7U,A0038036HJMMW88H5SCN,1.0,"It will not attach as said in instructions, wi...",1494633600,"It will not attach as said in instructions, wi...",4
...,...,...,...,...,...,...,...,...
1744394,True,B000V64P90,AZZYKX2KZ0Q82,5.0,I love the band and love the song. I keep it o...,1384819200,Loved it!,0
1744395,True,B00BOW41P8,AZZYKX2KZ0Q82,5.0,I don't get a lot of free time so the papers j...,1384819200,get it now,0
1744396,True,B0091LJ0IC,AZZYKX2KZ0Q82,5.0,I love martinis but have never actually used t...,1381190400,my favorite,0
1744397,True,B001KBYPWW,AZZYKX2KZ0Q82,5.0,I love them. They were what I was looking for....,1381190400,get what you pay for,0


In [29]:

train_df.groupby('reviewerID', as_index=False)['overall'].agg(['mean', 'count']).reset_index().sort_values(by=['mean', 'count'], ascending=[True, False])

,reviewerID,mean,count
90366,AMXE0HOAV0R9I,1.0,19
91512,AOJ9OER7G5Q3C,1.0,15
15023,A1KHTM1WYLVXAA,1.0,14
66355,A3I3OQF4FYR1SZ,1.0,14
81941,AB7W9FZ5O7CVJ,1.0,12
...,...,...,...
99819,AZQS2CHMYKQ6V,5.0,7
99820,AZQY5M5WGD7MB,5.0,7
99859,AZT2XCDE2SGUL,5.0,7
99890,AZUU1U55E1A3A,5.0,7


In [35]:
# join_df = pd.merge(train_df, all_beauty_df, how="left", on=['asin'])
# join_df.loc[join_df['main_cat'].isna() == False]
metadata_df = pd.DataFrame()
fileNames = os.listdir("../Data/Metadata")
with tqdm(total=len(fileNames), desc="Processing", leave=True, unit_scale=True) as pbar:
    for fileName in fileNames:
        primary_category = fileName.split(".")[0]
        temp_df = pd.read_csv("../Data/Metadata/" + fileName)
        temp_df['primary_cat'] = primary_category
        metadata_df = pd.concat([metadata_df, temp_df])
        pbar.update()
metadata_df.reset_index(inplace=True, drop=True)

Processing: 100%|██████████| 30.0/30.0 [00:36<00:00, 1.22s/it]


In [58]:
metadata_df.drop_duplicates(inplace=True)
metadata_df.reset_index(drop=True, inplace=True)

In [63]:
join_df = pd.merge(train_df, metadata_df, how="left", on='asin')
join_df
# train_df.join(metadata_df.set_index('asin'), on='asin')
# metadata_df.asin.value_counts(ascending=False)

,verified,asin,reviewerID,overall,reviewText,unixReviewTime,summary,vote,main_cat,title,primary_cat
0,True,B00UO1PFQO,A0038036HJMMW88H5SCN,1.0,"It will not attach as said in instructions, wi...",1494633600,"It will not attach as said in instructions, wi...",4,automotive,Husky Liners Under Seat Storage Box Fits 15-18...,automotive
1,True,B00WA4AM7U,A0038036HJMMW88H5SCN,5.0,"Great fit, like very much!",1494633600,Five Stars,0,automotive,Husky Liners 2nd Seat Floor Liner Fits 15-18 F...,automotive
2,True,B00WA4AM7U,A0038036HJMMW88H5SCN,5.0,"Great fit, like very much!",1494633600,Five Stars,0,automotive,Husky Liners 2nd Seat Floor Liner Fits 15-18 F...,automotive
3,True,B00WA4AM7U,A0038036HJMMW88H5SCN,5.0,"Great fit, like very much!",1494633600,Five Stars,0,automotive,Husky Liners 2nd Seat Floor Liner Fits 15-18 F...,automotive
4,True,B00WA4AM7U,A0038036HJMMW88H5SCN,1.0,"It will not attach as said in instructions, wi...",1494633600,"It will not attach as said in instructions, wi...",4,automotive,Husky Liners 2nd Seat Floor Liner Fits 15-18 F...,automotive
...,...,...,...,...,...,...,...,...,...,...,...
1744394,True,B000V64P90,AZZYKX2KZ0Q82,5.0,I love the band and love the song. I keep it o...,1384819200,Loved it!,0,NaN,NaN,NaN
1744395,True,B00BOW41P8,AZZYKX2KZ0Q82,5.0,I don't get a lot of free time so the papers j...,1384819200,get it now,0,NaN,NaN,NaN
1744396,True,B0091LJ0IC,AZZYKX2KZ0Q82,5.0,I love martinis but have never actually used t...,1381190400,my favorite,0,amazon home,Rabbit Electric Cocktail Mixer (18-Ounce),amazon product
1744397,True,B001KBYPWW,AZZYKX2KZ0Q82,5.0,I love them. They were what I was looking for....,1381190400,get what you pay for,0,amazon home,Yamazaki Stainless Steel My Chopsticks in Silv...,amazon product


In [66]:
len(join_df.loc[join_df['main_cat'].isna() == True]['asin'].unique().tolist())

10422

## Sample Data

### Single Process

In [9]:
def sampleData(sampleSize:int, totalSize:int, dataPath:str, outputPath:str):
    """function used to sample a small dataset from a large csv file

    Args:
        sampleSize (int): the size of the sample
        totalSize (int): the size of the original dataset
        dataPath (str): original dataset path
        outputPath (str): sample dataset output path
    """
    sampleIndicesList = sorted(random.sample(range(totalSize), sampleSize))
    sampleIndex = 0
    with open(dataPath, encoding="utf-8", newline='') as file:
        csvReader = csv.reader(file)
        csvHeader = next(csvReader) # read header
        with open(outputPath, "w", newline='') as outputFile:
            csvWriter = csv.writer(outputFile, csvHeader)
            csvWriter.writerow(csvHeader) # write header
            with tqdm(total=sampleSize, desc="Processing", leave=True, unit_scale=True) as pbar:
                for index, row in enumerate(csvReader):
                    if index == sampleIndicesList[sampleIndex]:
                        csvWriter.writerow(row)
                        sampleIndex += 1
                        pbar.update()
                    if sampleIndex == sampleSize:
                        break
                    

data_dir = "../Data/"
input_csv_path = os.path.join(data_dir, "All_Amazon_Review_User_Item_Rating.csv")
output_csv_path = os.path.join(data_dir, "sampled_data.csv")
total_records = 157260921
required_records = 2500000
sampleData(required_records, total_records, input_csv_path, output_csv_path)

Processing: 100%|██████████| 2.50M/2.50M [01:53<00:00, 22.0kit/s]


### Multi-process

In [4]:
import multiprocess_methods

if __name__ == '__main__':
    data_dir = "../Data/"
    input_csv_path = os.path.join(data_dir, "All_Amazon_Review_User_Item_Rating.csv")
    output_csv_path = os.path.join(data_dir, "sampled_data.csv")
    total_records = 157260921
    required_records = 2500000
    printParameters = True
    multiprocess_methods.sampleDataset(input_csv_path, total_records, output_csv_path, required_records, printParameters)

Dataset File: ../Data/All_Amazon_Review_User_Item_Rating.csv
Total Size: 157260921
Output File: ../Data/sampled_data.csv
Sample size: 2500000


Processing: 100%|██████████| 2.50M/2.50M [01:59<00:00, 20.8kit/s]


### Sample data directly from JSON

In [3]:
def sampleDataJSON(sampleSize:int, totalSize:int, dataPath:str, outputPath:str, columns:list):
    """function used to sample a small csv dataset from a large JSON file

    Args:
        sampleSize (int): the size of the sample
        totalSize (int): the size of the original dataset
        dataPath (str): original dataset path
        outputPath (str): sample dataset output path
        columns (list): list of cloumn required in the sampled dataset
    """
    sampleIndicesList = sorted(random.sample(range(totalSize), sampleSize))
    sampleIndex = 0

    with open(outputPath, "w", newline="") as outputFile:
        writer = csv.DictWriter(outputFile, columns)
        writer.writeheader()
        with open(dataPath, encoding="utf-8") as file:
            with tqdm(total=sampleSize, desc="Processing", leave=True, unit_scale=True) as pbar:
                for index, line in enumerate(file):
                    if index == sampleIndicesList[sampleIndex]:
                        row = json.loads(line)
                        writer.writerow(dict([(key, row.get(key, "")) for key in columns]))
                        sampleIndex += 1
                        pbar.update()
                    if sampleIndex == sampleSize:
                        break
                    

data_dir = "../Data/"
input_json_path = os.path.join(data_dir, "All_Amazon_Review_5.json")
output_csv_path = os.path.join(data_dir, "sampled_data.csv")
total_records = 157260921
required_records = 12500000
columns = ['verified', 'image', 'style', 'asin', 'reviewerID', 'overall', 'reviewText', 'reviewTime', 'summary', 'vote']
sampleDataJSON(required_records, total_records, input_json_path, output_csv_path, columns)

Processing: 100%|██████████| 12.5M/12.5M [26:06<00:00, 7.98kit/s]  


In [2]:
df = pd.read_csv("../Data/sampled_data.csv", low_memory=False)
df.dropna(how="all", inplace=True)
df.drop_duplicates(inplace=True)
df.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 12477503 entries, 0 to 12499999
Data columns (total 10 columns):
 #   Column      Dtype  
---  ------      -----  
 0   verified    bool   
 1   image       object 
 2   style       object 
 3   asin        object 
 4   reviewerID  object 
 5   overall     float64
 6   reviewText  object 
 7   reviewTime  object 
 8   summary     object 
 9   vote        object 
dtypes: bool(1), float64(1), object(8)
memory usage: 963.9+ MB


### Train - Validation - Test Split

In [6]:
data_dir = "../Data"

# Shuffle entire DataFrame and reset index before splitting
df = df.sample(frac=1).reset_index(drop=True)
total_records = len(df)

# Take first 60% of rows as training data
df_train = df.iloc[ : int(total_records * 0.6), :]
df_train.reset_index(inplace=True, drop=True)
df_train.to_csv(os.path.join(data_dir, "train.csv"))

# Take next 20% of rows as validation data
df_validation = df.iloc[int(total_records * 0.6) : int(total_records * 0.8), :]
df_validation.reset_index(inplace=True, drop=True)
df_validation.to_csv(os.path.join(data_dir, "validation.csv"))

# Take the last 20% of rows as test data
df_test = df.iloc[int(total_records * 0.8) : , :]
df_test.reset_index(inplace=True, drop=True)
df_test.to_csv(os.path.join(data_dir, "test.csv"))